In [3]:
import pandas as pd

# Load data dari CSV
df = pd.read_csv('surabaya_routes.csv')

# Menampilkan 5 baris pertama untuk melihat struktur data
print(df.head())


       Start_Name  Start_Lat   Start_Lon                 End_Name   End_Lat  \
0  Stasiun Gubeng  -7.257472  112.752088          Tunjungan Plaza -7.266735   
1  Stasiun Gubeng  -7.257472  112.752088  Kebun Binatang Surabaya -7.290215   
2  Stasiun Gubeng  -7.257472  112.752088     Surabaya Town Square -7.290116   
3  Stasiun Gubeng  -7.257472  112.752088          Grand City Mall -7.261414   
4  Stasiun Gubeng  -7.257472  112.752088      Terminal Bungurasih -7.345473   

      End_Lon  Distance_km  Duration_min  
0  112.736115       4.5955      6.468333  
1  112.735337       5.7728      6.386667  
2  112.734558       5.8817      6.803333  
3  112.751639       0.8897      1.513333  
4  112.742177      14.0382     15.553333  


In [4]:
import numpy as np
import random

# Menghitung jumlah state dan action
state_space = len(df)
action_space = len(df)

# Inisialisasi Q-table
Q_table = np.zeros((state_space, action_space))

# Hyperparameters
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.1  # Eksplorasi vs Eksploitasi

# Fungsi untuk memilih aksi menggunakan epsilon-greedy
def choose_action(state):
    if random.uniform(0, 1) < epsilon:
        return random.choice(range(action_space))  # Eksplorasi
    else:
        return np.argmax(Q_table[state])  # Eksploitasi

# Fungsi untuk menghitung reward
def calculate_reward(action):
    distance = df.iloc[action]['Distance_km']
    duration = df.iloc[action]['Duration_min']
    return - (distance + duration)  # Reward negatif untuk jarak dan durasi

# Training Q-Learning
for episode in range(1000):  # Jumlah episode (latihan)
    state = random.randint(0, state_space - 1)  # Pilih state acak sebagai titik mulai
    done = False

    while not done:
        action = choose_action(state)
        reward = calculate_reward(action)
        next_state = action  # Pindah ke state berikutnya setelah aksi

        # Pembaruan Q-table
        Q_table[state, action] = Q_table[state, action] + learning_rate * (reward + discount_factor * np.max(Q_table[next_state]) - Q_table[state, action])
        
        state = next_state
        done = True  # Setiap episode selesai setelah satu langkah

# Melihat hasil Q-table setelah training
print("Q-table:")
print(Q_table)


Q-table:
[[-1.10638333 -1.21594667 -1.26850333 ...  0.          0.
   0.        ]
 [-1.10638333 -1.21594667 -1.26850333 ...  0.          0.
   0.        ]
 [-1.10638333 -1.21594667 -1.26850333 ...  0.          0.
   0.        ]
 ...
 [-1.10638333 -1.21594667 -2.41015633 ...  0.          0.
  -2.38285   ]
 [-1.10638333 -1.21594667 -1.26850333 ...  0.          0.
   0.        ]
 [-2.10212833 -1.21594667 -1.26850333 ...  0.          0.
   0.        ]]


In [5]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Memilih fitur Distance_km untuk input (X) dan Duration_min untuk output (y)
X = df[['Distance_km']].values
y = df['Duration_min'].values

# Normalisasi data untuk LSTM
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Reshaping untuk input LSTM: [samples, timesteps, features]
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Membuat model LSTM
model = Sequential()
model.add(LSTM(units=50, activation='relu', input_shape=(X_scaled.shape[1], X_scaled.shape[2])))
model.add(Dense(1))  # Output: prediksi waktu tempuh
model.compile(optimizer='adam', loss='mean_squared_error')

# Melatih model LSTM
model.fit(X_scaled, y_scaled, epochs=100, batch_size=32)

# Prediksi waktu tempuh berdasarkan input Distance_km
predicted_time = model.predict(X_scaled)
predicted_time = scaler_y.inverse_transform(predicted_time)

# Menampilkan hasil prediksi
print("Prediksi Waktu Tempuh (Durasi):")
print(predicted_time[:5])


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.24453
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2487
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 988us/step - loss: 0.2346
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2185  
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2134
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1999
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1847
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1968
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 926us/step - loss: 0.1853
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1803
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1770
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1678 
Epoch 13/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.1543
Epoch 14/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1655
Epoch 15/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1532 
Epoch 16/100
3/3 ━━━━━━━

In [6]:
# Import Libraries
import numpy as np
import random
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler

# Simulasi data rute
df = pd.read_csv('surabaya_routes_clean.csv')

# Normalisasi Data
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

X = df[['Distance_km']].values
y = df['Duration_min'].values

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Reshape data untuk LSTM (samples, timesteps, features)
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# LSTM Model untuk memprediksi waktu tempuh
model_lstm = Sequential()
model_lstm.add(LSTM(units=50, activation='relu', input_shape=(X_scaled.shape[1], X_scaled.shape[2])))
model_lstm.add(Dense(1))  # Output: prediksi waktu tempuh
model_lstm.compile(optimizer='adam', loss='mean_squared_error')

# Melatih Model LSTM
model_lstm.fit(X_scaled, y_scaled, epochs=100, batch_size=32)

# Prediksi waktu tempuh menggunakan LSTM
predicted_time = model_lstm.predict(X_scaled)
predicted_time = scaler_y.inverse_transform(predicted_time)

# Definisikan Q-learning untuk optimasi rute

# Menghitung jumlah state dan action
state_space = len(df)
action_space = len(df)

# Inisialisasi Q-table
Q_table = np.zeros((state_space, action_space))

# Hyperparameters untuk Q-learning
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.1  # Eksplorasi vs Eksploitasi

# Fungsi untuk memilih aksi menggunakan epsilon-greedy
def choose_action(state):
    if random.uniform(0, 1) < epsilon:
        return random.choice(range(action_space))  # Eksplorasi
    else:
        return np.argmax(Q_table[state])  # Eksploitasi

# Fungsi untuk menghitung reward
def calculate_reward(action):
    # Menggunakan prediksi waktu tempuh dari LSTM sebagai reward
    distance = df.iloc[action]['Distance_km']
    duration_predicted = predicted_time[action][0]  # Menggunakan hasil prediksi LSTM
    return - (distance + duration_predicted)  # Reward negatif agar semakin kecil waktu dan jarak semakin tinggi reward

# Training Q-learning
for episode in range(1000):  # Jumlah episode (latihan)
    state = random.randint(0, state_space - 1)  # Pilih state acak sebagai titik mulai
    done = False

    while not done:
        action = choose_action(state)
        reward = calculate_reward(action)
        next_state = action  # Pindah ke state berikutnya setelah aksi

        # Pembaruan Q-table
        Q_table[state, action] = Q_table[state, action] + learning_rate * (reward + discount_factor * np.max(Q_table[next_state]) - Q_table[state, action])
        
        state = next_state
        done = True  # Setiap episode selesai setelah satu langkah

# Melihat hasil Q-table setelah training
print("Q-table:")
print(Q_table)

# Menggunakan Q-table untuk memilih rute terbaik
best_route = np.argmax(Q_table[0])  # Memilih aksi terbaik dari state 0 (mulai dari titik awal)
print(f"Rute terbaik yang dipilih berdasarkan Q-learning: Rute {best_route}")



Epoch 1/100


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2450
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2368
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2378  
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2350 
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 815us/step - loss: 0.2137
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.2244
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2022 
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2083 
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.1953
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1814
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1777
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1847  
Epoch 13/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1838
Epoch 14/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1751 
Epoch 15/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1604  
Epoch 16/100
3/3 ━━━━━━━━━━━━━━━━